In [1]:
require 'nn';
require 'nngraph';

# Vh

* nInputPlane = 1
* nOutputPlane = scoreDepth
* height = L
* width = annotationDepth
* input = nInputPlane x height x width
* module = nn.SpatialConvolution(nInputPlane, nOutputPlane, kW, kH, [dW], [dH], [padW], [padH])
* kW = annotationDepth
* kH = 1

In [8]:
annotationDepth = 5
scoreDepth = 4
L = 10

m = nn.SpatialConvolutionMM(1,scoreDepth,annotationDepth,1)
p,dp = m:getParameters()
p:fill(1)

h = torch.ones(1,L,annotationDepth)
Vh = m:forward(h)
print(Vh:size())
print(Vh:resize(scoreDepth,L))


  4
 10
  1
[torch.LongStorage of size 3]

 6  6  6  6  6  6  6  6  6  6
 6  6  6  6  6  6  6  6  6  6
 6  6  6  6  6  6  6  6  6  6
 6  6  6  6  6  6  6  6  6  6
[torch.DoubleTensor of size 4x10]



* inputFrameSize = annotationDepth
* nOutputPlane = scoreDepth
* input = L x inputFrameSize
* module = nn.TemporalConvolution(inputFrameSize, outputFrameSize, kW, [dW])
* kW = 1

In [66]:
annotationDepth = 5
scoreDepth = 4
L = 10

m = nn.TemporalConvolution(annotationDepth,scoreDepth,1)
w = m:parameters()[1]
b = m:parameters()[2]
i=0
w:apply(function() i=i+1 return i end)
b:fill(0)

h = torch.ones(L,annotationDepth)
Vh = m:forward(h)
print(Vh:size())

h = torch.ones(8,L,annotationDepth)
Vh = m:forward(h)
print(Vh:size())


 10
  4
[torch.LongStorage of size 2]


  8
 10
  4
[torch.LongStorage of size 3]



In [29]:
Vh

 15  40  65  90
 15  40  65  90
 15  40  65  90
 15  40  65  90
 15  40  65  90
 15  40  65  90
 15  40  65  90
 15  40  65  90
 15  40  65  90
 15  40  65  90
[torch.DoubleTensor of size 10x4]



In [31]:
w

  1   2   3   4   5
  6   7   8   9  10
 11  12  13  14  15
 16  17  18  19  20
[torch.DoubleTensor of size 4x5]



# UF

* module = nn.Padding(dim, pad [, nInputDim, value])

In [43]:
hybridAttendFilterSize = 3
hybridAttendFeatureMaps = 5
L = 10

--print(nn.SpatialZeroPadding(0,0,1,1):forward(nn.Reshape(1,L,1):forward(torch.ones(1,10,1))):size())

print(nn.Padding(1,-2,2):forward(torch.ones(10,1)):size())
print(nn.Padding(1,2,2):forward(torch.ones(5,10,1)):size())
print(nn.Padding(1,2,2):forward(nn.Padding(1,-2,2):forward(torch.ones(10,1))))


 12
  1
[torch.LongStorage of size 2]


  5
 12
  1
[torch.LongStorage of size 3]

 0
 0
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 0
 0
[torch.DoubleTensor of size 14x1]



* inputFrameSize = 1
* nOutputPlane = hybridAttendFeatureMaps
* input = L x inputFrameSize
* module = nn.TemporalConvolution(inputFrameSize, outputFrameSize, kW, [dW])
* kW = hybridAttendFilterSize

### odd filter size

In [53]:
hybridAttendFilterSize = 3
hybridAttendFeatureMaps = 5
L = 10

pad_left = math.floor((hybridAttendFilterSize-1)/2)
pad_right = math.floor((hybridAttendFilterSize-1)/2)

nn.Padding(1,2,2):forward(nn.Padding(1,-2,2):forward(torch.ones(10,1)))

m = nn.Sequential()
m:add(nn.Padding(1,-pad_left,2))
m:add(nn.Padding(1,pad_right,2))
m:add(nn.TemporalConvolution(1,hybridAttendFeatureMaps,hybridAttendFilterSize))

x = torch.ones(10,1)
print(m:forward(x):size())

x = torch.ones(11,1)
print(m:forward(x):size())


 10
  5
[torch.LongStorage of size 2]


 11
  5
[torch.LongStorage of size 2]



## even filter size

In [54]:
hybridAttendFilterSize = 4
hybridAttendFeatureMaps = 5
scoreDepth = 4
L = 10

pad_left = math.floor((hybridAttendFilterSize)/2)
pad_right = math.floor((hybridAttendFilterSize)/2)-1

nn.Padding(1,2,2):forward(nn.Padding(1,-2,2):forward(torch.ones(10,1)))

m = nn.Sequential()
m:add(nn.Padding(1,-pad_left,2))
m:add(nn.Padding(1,pad_right,2))
m:add(nn.TemporalConvolution(1,hybridAttendFeatureMaps,hybridAttendFilterSize))

x = torch.ones(10,1)
print(m:forward(x):size())

x = torch.ones(11,1)
print(m:forward(x):size())


 10
  5
[torch.LongStorage of size 2]


 11
  5
[torch.LongStorage of size 2]



## We have calculated F, now do UF ... actually (UF)^T

* module = nn.TemporalConvolution(inputFrameSize, outputFrameSize, kW, [dW])
* input = L x inputFrameSize
#### F
* inputFrameSize = 1
* nOutputPlane = hybridAttendFeatureMaps
* kW = hybridAttendFilterSize
#### UF
* inputFrameSize = hybridAttendFeatureMaps
* nOutputPlane = scoreDepth
* kW = 1

In [58]:
hybridAttendFilterSize = 3
hybridAttendFeatureMaps = 5
scoreDepth = 4
L = 10

pad_left = math.floor((hybridAttendFilterSize-1)/2)
pad_right = math.floor((hybridAttendFilterSize-1)/2)

nn.Padding(1,2,2):forward(nn.Padding(1,-2,2):forward(torch.ones(10,1)))

m = nn.Sequential()
m:add(nn.Padding(1,-pad_left,2))
m:add(nn.Padding(1,pad_right,2))
m:add(nn.TemporalConvolution(1,hybridAttendFeatureMaps,hybridAttendFilterSize))
m:add(nn.TemporalConvolution(hybridAttendFeatureMaps,scoreDepth,1))

x = torch.ones(10,1)
print(m:forward(x):size())


 10
  6
[torch.LongStorage of size 2]



# Ws


* module = nn.TemporalConvolution(inputFrameSize, outputFrameSize, kW, [dW])
* input = L x inputFrameSize
* L = stateDepth
* inputFrameSize = 1
* nOutputPlane = scoreDepth
* kW = stateDepth

In [60]:
stateDepth = 5
scoreDepth = 4

m = nn.Sequential()
m:add(nn.TemporalConvolution(1,scoreDepth,stateDepth))

x = torch.ones(stateDepth,1)
print(m:forward(x):size())

x = torch.ones(8,stateDepth,1)
print(m:forward(x):size())


 1
 4
[torch.LongStorage of size 2]


 8
 1
 4
[torch.LongStorage of size 3]



#### replicate L times

module = nn.Replicate(nFeature [, dim, ndim])

In [64]:
x = torch.randn(1,4)
print(x)
print(nn.Reshape(6,4):forward(nn.Replicate(6,1,2):forward(x)))

 0.2889 -0.1294  1.4467  0.0965
[torch.DoubleTensor of size 1x4]

 0.2889 -0.1294  1.4467  0.0965
 0.2889 -0.1294  1.4467  0.0965
 0.2889 -0.1294  1.4467  0.0965
 0.2889 -0.1294  1.4467  0.0965
 0.2889 -0.1294  1.4467  0.0965
 0.2889 -0.1294  1.4467  0.0965
[torch.DoubleTensor of size 6x4]



In [65]:
stateDepth = 5
scoreDepth = 4
L = 10

m = nn.Sequential()
m:add(nn.TemporalConvolution(1,scoreDepth,stateDepth))
m:add(nn.Replicate(L,1,2))
m:add(nn.Reshape(L,scoreDepth))

x = torch.ones(stateDepth,1)
print(m:forward(x):size())

x = torch.ones(8,stateDepth,1)
print(m:forward(x):size())


 10
  4
[torch.LongStorage of size 2]


  8
 10
  4
[torch.LongStorage of size 3]



# add bias to input to tanh(Z+b)

In [104]:
L = 10
scoreDepth = 4
batchSize = 2

Z = torch.ones(batchSize,L,scoreDepth)

bias = torch.randn(scoreDepth)
ones = torch.ones(Z:size(2))
bias_grid = torch.Tensor():resizeAs(Z[1]):fill(0):addr(1, ones, bias)
--bias_grid:expandAs(Z)

print(bias_grid:resize(1,Z:size(2),Z:size(3)):expandAs(Z))


(1,.,.) = 
  0.3949 -0.9719 -0.2226 -0.0188
  0.3949 -0.9719 -0.2226 -0.0188
  0.3949 -0.9719 -0.2226 -0.0188
  0.3949 -0.9719 -0.2226 -0.0188
  0.3949 -0.9719 -0.2226 -0.0188
  0.3949 -0.9719 -0.2226 -0.0188
  0.3949 -0.9719 -0.2226 -0.0188
  0.3949 -0.9719 -0.2226 -0.0188
  0.3949 -0.9719 -0.2226 -0.0188
  0.3949 -0.9719 -0.2226 -0.0188

(2,.,.) = 
  0.3949 -0.9719 -0.2226 -0.0188
  0.3949 -0.9719 -0.2226 -0.0188
  0.3949 -0.9719 -0.2226 -0.0188
  0.3949 -0.9719 -0.2226 -0.0188
  0.3949 -0.9719 -0.2226 -0.0188
  0.3949 -0.9719 -0.2226 -0.0188
  0.3949 -0.9719 -0.2226 -0.0188
  0.3949 -0.9719 -0.2226 -0.0188
  0.3949 -0.9719 -0.2226 -0.0188
  0.3949 -0.9719 -0.2226 -0.0188
[torch.DoubleTensor of size 2x10x4]



In [1]:
require 'nn'
require 'AddBias'

In [2]:
L = 10
scoreDepth = 4
batchSize = 2

m = nn.AddBias(scoreDepth,false)

Z = torch.ones(L,scoreDepth)
print(m:forward(Z))

Z = torch.ones(batchSize,L,scoreDepth)
print(m:forward(Z))

 0.5298  0.5552  1.0446  0.5357
 0.5298  0.5552  1.0446  0.5357
 0.5298  0.5552  1.0446  0.5357
 0.5298  0.5552  1.0446  0.5357
 0.5298  0.5552  1.0446  0.5357
 0.5298  0.5552  1.0446  0.5357
 0.5298  0.5552  1.0446  0.5357
 0.5298  0.5552  1.0446  0.5357
 0.5298  0.5552  1.0446  0.5357
 0.5298  0.5552  1.0446  0.5357
[torch.DoubleTensor of size 10x4]

(1,.,.) = 
  0.5298  0.5552  1.0446  0.5357
  0.5298  0.5552  1.0446  0.5357
  0.5298  0.5552  1.0446  0.5357
  0.5298  0.5552  1.0446  0.5357
  0.5298  0.5552  1.0446  0.5357
  0.5298  0.5552  1.0446  0.5357
  0.5298  0.5552  1.0446  0.5357
  0.5298  0.5552  1.0446  0.5357
  0.5298  0.5552  1.0446  0.5357
  0.5298  0.5552  1.0446  0.5357

(2,.,.) = 
  0.5298  0.5552  1.0446  0.5357
  0.5298  0.5552  1.0446  0.5357
  0.5298  0.5552  1.0446  0.5357
  0.5298  0.5552  1.0446  0.5357
  0.5298  0.5552  1.0446  0.5357
  0.5298  0.5552  1.0446  0.5357
  0.5298  0.5552  1.0446  0.5357
  0.5298  0.5552  1.0446  0.5357
  0.5298  0.5552  1.0446  0.

In [8]:
L = 9
scoreDepth = 4
batchSize = 3

m = nn.AddBias(scoreDepth,false)
p,dp = m:getParameters()

m:zeroGradParameters()
Z = torch.ones(L,scoreDepth)
m:forward(Z)
m:backward(Z,Z)
print(dp)

m:zeroGradParameters()
Z = torch.ones(batchSize,L,scoreDepth)
m:forward(Z)
m:backward(Z,Z)
print(dp)

 9
 9
 9
 9
[torch.DoubleTensor of size 4]

 27
 27
 27
 27
[torch.DoubleTensor of size 4]



# e_t

* module = nn.TemporalConvolution(inputFrameSize, outputFrameSize, kW, [dW])
* input = L x inputFrameSize
* inputFrameSize = scoreDepth
* nOutputPlane = 1
* kW = scoreDepth

In [25]:
scoreDepth = 4
L = 10
batchSize = 8

m = nn.Sequential()
m:add(nn.TemporalConvolution(scoreDepth,1,1))

x = torch.ones(L,scoreDepth)
print(m:forward(x):size())

x = torch.ones(batchSize,L,scoreDepth)
print(m:forward(x):size())


 10
  1
[torch.LongStorage of size 2]


  8
 10
  1
[torch.LongStorage of size 3]



# alpha

In [36]:
L = 10
batchSize = 8

m = nn.Sequential()
m:add(nn.Reshape(L))
m:add(nn.SoftMax())

e = torch.randn(L,1)
print(m:forward(e))

e = torch.randn(batchSize,L,1)
print(m:forward(e))

 0.1876
 0.0429
 0.0349
 0.0166
 0.1256
 0.0318
 0.0867
 0.1422
 0.1153
 0.2165
[torch.DoubleTensor of size 10]

 0.1207  0.0105  0.3167  0.1983  0.0944  0.0229  0.0645  0.0531  0.0356  0.0833
 0.1076  0.1684  0.0765  0.3354  0.0376  0.0366  0.0160  0.0203  0.0839  0.1178
 0.1586  0.0483  0.0071  0.0304  0.0805  0.0524  0.0232  0.2953  0.0432  0.2609
 0.0759  0.0856  0.0637  0.0380  0.2950  0.0699  0.0438  0.1340  0.0080  0.1861
 0.1196  0.1007  0.0803  0.1347  0.1056  0.1594  0.0205  0.1102  0.1072  0.0618
 0.0295  0.0583  0.0845  0.0745  0.0453  0.2239  0.2827  0.0532  0.0698  0.0781
 0.1176  0.0311  0.1571  0.0354  0.0123  0.0875  0.3124  0.0516  0.1290  0.0660
 0.0550  0.0396  0.0650  0.0101  0.5254  0.0365  0.0564  0.0132  0.1088  0.0899
[torch.DoubleTensor of size 8x10]



# c

In [41]:
L = 10
annotationDepth = 5
batchSize = 8

m = nn.MM()

h = torch.ones(L,annotationDepth)
alpha = torch.ones(1,L)
print(m:forward({alpha,h}))

h = torch.ones(batchSize,L,annotationDepth)
alpha = torch.ones(batchSize,1,L)
print(m:forward({alpha,h}))

 10  10  10  10  10
[torch.DoubleTensor of size 1x5]

(1,.,.) = 
  10  10  10  10  10

(2,.,.) = 
  10  10  10  10  10

(3,.,.) = 
  10  10  10  10  10

(4,.,.) = 
  10  10  10  10  10

(5,.,.) = 
  10  10  10  10  10

(6,.,.) = 
  10  10  10  10  10

(7,.,.) = 
  10  10  10  10  10

(8,.,.) = 
  10  10  10  10  10
[torch.DoubleTensor of size 8x1x5]



# RNN with nonrecurrent inputs

#### First, test to make sure LSTM still works with it

In [1]:
require 'nn';
require 'RNN2';
require 'RNN';
require 'LSTM';

In [2]:
diminput = 4
dimoutput = 3
T = 10
batchSize = 4
peepholes = false
lstm1 = nn.LSTM(diminput,dimoutput,peepholes)
rnn1 = nn.RNN(lstm1,T)

lstm2 = nn.LSTM(diminput,dimoutput,peepholes)
rnn2 = nn.RNN2(lstm2,T)

./RNN2.lua:10: recurrent must specify dimoutput
stack traceback:
	[C]: in function 'assert'
	./RNN2.lua:10: in function '__init'
	...ustinmaojones/torch/install/share/lua/5.1/torch/init.lua:54: in function <...ustinmaojones/torch/install/share/lua/5.1/torch/init.lua:50>
	[C]: in function 'RNN2'
	[string "diminput = 4..."]:10: in main chunk
	[C]: in function 'xpcall'
	...stinmaojones/torch/install/share/lua/5.1/itorch/main.lua:179: in function <...stinmaojones/torch/install/share/lua/5.1/itorch/main.lua:143>
	...stinmaojones/torch/install/share/lua/5.1/lzmq/poller.lua:75: in function 'poll'
	...nmaojones/torch/install/share/lua/5.1/lzmq/impl/loop.lua:307: in function 'poll'
	...nmaojones/torch/install/share/lua/5.1/lzmq/impl/loop.lua:325: in function 'sleep_ex'
	...nmaojones/torch/install/share/lua/5.1/lzmq/impl/loop.lua:370: in function 'start'
	...stinmaojones/torch/install/share/lua/5.1/itorch/main.lua:350: in main chunk
	[C]: in function 'require'
	[string "arg={'/Users/justinmaojon

In [3]:
rnn2.gradRecurrentInput

[string "local f = function() return rnn2.gradRecurren..."]:1: attempt to index global 'rnn2' (a nil value)
stack traceback:
	[string "local f = function() return rnn2.gradRecurren..."]:1: in function 'f'
	[string "local f = function() return rnn2.gradRecurren..."]:1: in main chunk
	[C]: in function 'xpcall'
	...stinmaojones/torch/install/share/lua/5.1/itorch/main.lua:179: in function <...stinmaojones/torch/install/share/lua/5.1/itorch/main.lua:143>
	...stinmaojones/torch/install/share/lua/5.1/lzmq/poller.lua:75: in function 'poll'
	...nmaojones/torch/install/share/lua/5.1/lzmq/impl/loop.lua:307: in function 'poll'
	...nmaojones/torch/install/share/lua/5.1/lzmq/impl/loop.lua:325: in function 'sleep_ex'
	...nmaojones/torch/install/share/lua/5.1/lzmq/impl/loop.lua:370: in function 'start'
	...stinmaojones/torch/install/share/lua/5.1/itorch/main.lua:350: in main chunk
	[C]: in function 'require'
	[string "arg={'/Users/justinmaojones/Library/Jupyter/r..."]:1: in main chunk: 

In [4]:
p1,dp1 = rnn1:getParameters()
p2,dp2 = rnn2:getParameters()

dp1:zero();
dp2:zero()

p2:copy(p1)

if batchSize == 1 then
    x = torch.randn(T,diminput)
    dy = torch.randn(T,dimoutput)
else
    x = torch.randn(batchSize,T,diminput)
    dy = torch.randn(batchSize,T,dimoutput)
end

f1 = rnn1:forward(x)
b1 = rnn1:backward(x,dy)

f2 = rnn2:forward(x)
b2 = rnn2:backward(x,dy)

In [5]:
(f1-f2):norm()

0	


In [6]:
(b1-b2):norm()

0	


# Attention!

In [1]:
require 'nn';
require 'Attention';
require 'LSTM';
require 'nngraph';
nngraph.setDebug(true)

In [2]:
diminput = 4
dimoutput = 3
scoreDepth = 5
hybridAttendFilterSize = 3
hybridAttendFeatureMaps = 4
stateDepth = 4
annotationDepth = 6
outputDepth = 2
L = 10
T = 9
batchSize = 4
peepholes = false
mlpDepth = 7


dec_rec_inp = nn.Identity()()
inp, prev_s, prev_mem = dec_rec_inp:split(3)
c, prev_y = inp:split(2)

lstm_inp = nn.CAddTable()({nn.Linear(annotationDepth,stateDepth)(c),nn.Linear(outputDepth,stateDepth)(prev_y)})
lstm = nn.LSTM(stateDepth,stateDepth,peepholes)({lstm_inp,prev_s,prev_mem})

nngraph.annotateNodes()
decoder_recurrent = nn.gModule({dec_rec_inp},{lstm})
decoder_recurrent.name = "decoder_recurrent"

dec_mlp_inp = nn.Identity()()
mlp_inp = nn.JoinTable(1,1)(dec_mlp_inp)
mlp     = nn.Sequential()
mlp:add(nn.Linear(stateDepth+annotationDepth,mlpDepth))
mlp:add(nn.ReLU())
mlp:add(nn.Linear(mlpDepth,outputDepth))
mlp:add(nn.LogSoftMax())
decoder_mlp = nn.gModule({dec_mlp_inp},{mlp(mlp_inp)})
decoder_mlp.name = "decoder_mlp"


m = nn.Attention(decoder_recurrent,decoder_mlp,scoreDepth,hybridAttendFilterSize,hybridAttendFeatureMaps,stateDepth,annotationDepth,outputDepth,L,T)


In [3]:
h = torch.ones(L,annotationDepth)
m:forward(h)
m:backward(h,torch.ones(T,outputDepth))

RNN3 type(input) ==	table	
RNN3 input = 	{
  1 : DoubleTensor - size: 10x5
  2 : DoubleTensor - size: 10x6
}
==================================> RNN t =	1	
Recurrent type(inp) ==	table	
Recurrent inp = 	{
  1 : DoubleTensor - size: 10x5
  2 : DoubleTensor - size: 10x6
}
Recurrent	
inp	{
  1 : DoubleTensor - size: 10x5
  2 : DoubleTensor - size: 10x6
}
prev_y	 0
 0
[torch.DoubleTensor of size 2]

prev_h	{
  1 : DoubleTensor - size: 10
  2 : DoubleTensor - size: 4
  3 : DoubleTensor - size: 4
}


RNN3 output	
y	-0.8086
-0.5897
[torch.DoubleTensor of size 2]

h	{
  1 : DoubleTensor - size: 10
  2 : DoubleTensor - size: 4
  3 : DoubleTensor - size: 4
}
==================================> RNN t =	2	
Recurrent type(inp) ==	table	
Recurrent inp = 	{
  1 : DoubleTensor - size: 10x5
  2 : DoubleTensor - size: 10x6
}
Recurrent	
inp	{
  1 : DoubleTensor - size: 10x5
  2 : DoubleTensor - size: 10x6
}
prev_y	-0.8086
-0.5897
[torch.DoubleTensor of size 2]

prev_h	{
  1 : DoubleTensor - size: 10
  2 : DoubleTensor - size: 4
  3 : DoubleTensor - size: 4
}
RNN3 output	
y	-0.8141
-0.5853
[torch.DoubleTensor of size 2]

h	{
  1 : DoubleTensor - size: 10
  2 : DoubleTensor - size: 4
  3 : DoubleTensor - size: 4
}
==================================> RNN t =	3	
Recurrent type(inp) ==	table	
Recurrent inp = 	{
  1 : DoubleTensor - size: 10x5
  2 : DoubleTensor - size: 10x6
}
Recurrent	
inp	{
  1 : DoubleTensor - size: 10x5
  2 : DoubleTensor - size: 10x6
}
prev_y	-0.8141
-0.5853
[torch.DoubleTensor

RNN3 output	
y	-0.8176
-0.5825
[torch.DoubleTensor of size 2]

h	{
  1 : DoubleTensor - size: 10
  2 : DoubleTensor - size: 4
  3 : DoubleTensor - size: 4
}
==================================> RNN t =	4	
Recurrent type(inp) ==	table	
Recurrent inp = 	{
  1 : DoubleTensor - size: 10x5
  2 : DoubleTensor - size: 10x6
}
Recurrent	
inp	{
  1 : DoubleTensor - size: 10x5
  2 : DoubleTensor - size: 10x6
}
prev_y	-0.8176
-0.5825
[torch.DoubleTensor of size 2]

prev_h	{
  1 : DoubleTensor - size: 10
  2 : DoubleTensor - size: 4
  3 : DoubleTensor - size: 4
}
RNN3 output	
y	-0.8199
-0.5807
[torch.DoubleTensor of size 2]

h	{
  1 : DoubleTensor - size: 10
  2 : DoubleTensor - size: 4
  3 : DoubleTensor - size: 4
}
==================================> RNN t =	5	
Recurrent type(inp) ==	table	
Recurrent inp = 	{
  1 : DoubleTensor - size: 10x5
  2 : DoubleTensor - size: 10x6
}
Recurrent	


inp	{
  1 : DoubleTensor - size: 10x5
  2 : DoubleTensor - size: 10x6
}
prev_y	-0.8199
-0.5807
[torch.DoubleTensor of size 2]

prev_h	{
  1 : DoubleTensor - size: 10
  2 : DoubleTensor - size: 4
  3 : DoubleTensor - size: 4
}
RNN3 output	
y	-0.8215
-0.5794
[torch.DoubleTensor of size 2]

h	{
  1 : DoubleTensor - size: 10
  2 : DoubleTensor - size: 4
  3 : DoubleTensor - size: 4
}
==================================> RNN t =	6	
Recurrent type(inp) ==	table	
Recurrent inp = 	{
  1 : DoubleTensor - size: 10x5
  2 : DoubleTensor - size: 10x6
}
Recurrent	
inp	{
  1 : DoubleTensor - size: 10x5
  2 : DoubleTensor - size: 10x6
}
prev_y	-0.8215
-0.5794
[torch.DoubleTensor of size 2]

prev_h	{
  1 : DoubleTensor - size: 10
  2 : DoubleTensor - size: 4
  3 : DoubleTensor - size: 4
}


RNN3 output	
y	-0.8227
-0.5785
[torch.DoubleTensor of size 2]

h	{
  1 : DoubleTensor - size: 10
  2 : DoubleTensor - size: 4
  3 : DoubleTensor - size: 4
}
==================================> RNN t =	7	
Recurrent type(inp) ==	table	
Recurrent inp = 	{
  1 : DoubleTensor - size: 10x5
  2 : DoubleTensor - size: 10x6
}
Recurrent	
inp	{
  1 : DoubleTensor - size: 10x5
  2 : DoubleTensor - size: 10x6
}
prev_y	-0.8227
-0.5785
[torch.DoubleTensor of size 2]

prev_h	{
  1 : DoubleTensor - size: 10
  2 : DoubleTensor - size: 4
  3 : DoubleTensor - size: 4
}
RNN3 output	
y	-0.8235
-0.5778
[torch.DoubleTensor of size 2]

h	{
  1 : DoubleTensor - size: 10
  2 : DoubleTensor - size: 4
  3 : DoubleTensor - size: 4
}
==================================> RNN t =	8	
Recurrent type(inp) ==	table	
Recurrent inp = 	{
  1 : DoubleTensor - size: 10x5
  2 : DoubleTensor - size: 10x6
}
Recurrent	
inp	{
  1 : DoubleTensor - size: 10x5
  2 : DoubleTensor - size: 10x6
}
prev_y	-0.8235
-0.5778
[torch.DoubleTensor

...

LSTM	
inp	 1.6938
-0.1170
 1.2906
 1.8176
[torch.DoubleTensor of size 4]

prev_h	-0.2171
 0.1783
-0.1736
-0.2119
[torch.DoubleTensor of size 4]

prev_c	-0.5116
 0.2976
-0.5282
-0.6176
[torch.DoubleTensor of size 4]

LSTM	
inp	 1.6935
-0.1168
 1.2906
 1.8172
[torch.DoubleTensor of size 4]

prev_h	-0.2157
 0.1629
-0.1683
-0.1979
[torch.DoubleTensor of size 4]

prev_c	-0.5060
 0.2716
-0.5063
-0.5653
[torch.DoubleTensor of size 4]



LSTM	
inp	 1.6932
-0.1164
 1.2907
 1.8166
[torch.DoubleTensor of size 4]

prev_h	-0.2118
 0.1436
-0.1600
-0.1811
[torch.DoubleTensor of size 4]

prev_c	-0.4937
 0.2393
-0.4738
-0.5059
[torch.DoubleTensor of size 4]

LSTM	
inp	 1.6927
-0.1158
 1.2908
 1.8159
[torch.DoubleTensor of size 4]

prev_h	

-0.2034
 0.1193
-0.1469
-0.1602
[torch.DoubleTensor of size 4]

prev_c	-0.4694
 0.1991
-0.4266
-0.4367
[torch.DoubleTensor of size 4]

LSTM	
inp	 1.6921
-0.1151
 1.2909
 1.8147
[torch.DoubleTensor of size 4]

prev_h	-0.1868
 0.0889
-0.1268
-0.1333
[torch.DoubleTensor of size 4]

prev_c	-0.4248
 0.1494
-0.3598
-0.3529
[torch.DoubleTensor of size 4]



LSTM	
inp	 1.6911
-0.1139
 1.2911
 1.8131
[torch.DoubleTensor of size 4]

prev_h	-0.1555
 0.0521
-0.0965
-0.0962
[torch.DoubleTensor of size 4]

prev_c	-0.3460
 0.0889
-0.2674
-0.2458
[torch.DoubleTensor of size 4]

LSTM	
inp	 1.6895
-0.1121
 1.2913
 1.8105
[torch.DoubleTensor of size 4]

prev_h	0.01 *
-9.2555
 0.9338
-5.3572
-4.5063
[torch.DoubleTensor of size 4]

prev_c	-0.2114
 0.0170
-0.1436
-0.0974
[torch.DoubleTensor of size 4]



LSTM	
inp	 1.6365
 0.1245
 0.9352
 1.3752
[torch.DoubleTensor of size 4]

prev_h	 0
 0
 0
 0
[torch.DoubleTensor of size 4]

prev_c	 0
 0
 0
 0
[torch.DoubleTensor of size 4]



In [4]:
h = torch.ones(batchSize,L,annotationDepth)
m:forward(h)
m:backward(h,torch.ones(batchSize,T,outputDepth))

RNN3 type(input) ==	table	
RNN3 input = 	{
  1 : DoubleTensor - size: 4x10x5
  2 : DoubleTensor - size: 4x10x6
}
==================================> RNN t =	1	
Recurrent type(inp) ==	table	
Recurrent inp = 	{
  1 : DoubleTensor - size: 4x10x5
  2 : DoubleTensor - size: 4x10x6
}
Recurrent	
inp	{
  1 : DoubleTensor - size: 4x10x5
  2 : DoubleTensor - size: 4x10x6
}
prev_y	 0  0
 0  0
 0  0
 0  0
[torch.DoubleTensor of size 4x2]

prev_h	{
  1 : DoubleTensor - size: 4x10
  2 : DoubleTensor - size: 4x4
  3 : DoubleTensor - size: 4x4
}


RNN3 output	
y	-0.8086 -0.5897
-0.8086 -0.5897
-0.8086 -0.5897
-0.8086 -0.5897
[torch.DoubleTensor of size 4x2]

h	{
  1 : DoubleTensor - size: 4x10
  2 : DoubleTensor - size: 4x4
  3 : DoubleTensor - size: 4x4
}
==================================> RNN t =	2	
Recurrent type(inp) ==	table	
Recurrent inp = 	{
  1 : DoubleTensor - size: 4x10x5
  2 : DoubleTensor - size: 4x10x6
}
Recurrent	
inp	{
  1 : DoubleTensor - size: 4x10x5
  2 : DoubleTensor - size: 4x10x6
}
prev_y	-0.8086 -0.5897
-0.8086 -0.5897
-0.8086 -0.5897
-0.8086 -0.5897
[torch.DoubleTensor of size 4x2]

prev_h	{
  1 : DoubleTensor - size: 4x10
  2 : DoubleTensor - size: 4x4
  3 : DoubleTensor - size: 4x4
}
RNN3 output	
y	-0.8141 -0.5853
-0.8141 -0.5853
-0.8141 -0.5853
-0.8141 -0.5853
[torch.DoubleTensor of size 4x2]

h	{


  1 : DoubleTensor - size: 4x10
  2 : DoubleTensor - size: 4x4
  3 : DoubleTensor - size: 4x4
}
==================================> RNN t =	3	
Recurrent type(inp) ==	table	
Recurrent inp = 	{
  1 : DoubleTensor - size: 4x10x5
  2 : DoubleTensor - size: 4x10x6
}
Recurrent	
inp	{
  1 : DoubleTensor - size: 4x10x5
  2 : DoubleTensor - size: 4x10x6
}
prev_y	-0.8141 -0.5853
-0.8141 -0.5853
-0.8141 -0.5853
-0.8141 -0.5853
[torch.DoubleTensor of size 4x2]

prev_h	{
  1 : DoubleTensor - size: 4x10
  2 : DoubleTensor - size: 4x4
  3 : DoubleTensor - size: 4x4
}
RNN3 output	
y	-0.8176 -0.5825
-0.8176 -0.5825
-0.8176 -0.5825
-0.8176 -0.5825
[torch.DoubleTensor of size 4x2]

h	{
  1 : DoubleTensor - size: 4x10
  2 : DoubleTensor - size: 4x4
  3 : DoubleTensor - size: 4x4
}
==================================> RNN t =	4	
Recurrent type(inp) ==	table	
Recurrent inp = 	{
  1 : DoubleTensor - size: 4x10x5
  2 : DoubleTensor - size: 4x10x6
}
Recurrent	
inp	{
  1 : DoubleTensor - size: 4x10x5
  2 : Doubl

RNN3 output	
y	-0.8199 -0.5807
-0.8199 -0.5807
-0.8199 -0.5807
-0.8199 -0.5807
[torch.DoubleTensor of size 4x2]

h	{
  1 : DoubleTensor - size: 4x10
  2 : DoubleTensor - size: 4x4
  3 : DoubleTensor - size: 4x4
}
==================================> RNN t =	5	
Recurrent type(inp) ==	table	
Recurrent inp = 	{
  1 : DoubleTensor - size: 4x10x5
  2 : DoubleTensor - size: 4x10x6
}
Recurrent	
inp	{
  1 : DoubleTensor - size: 4x10x5
  2 : DoubleTensor - size: 4x10x6
}
prev_y	-0.8199 -0.5807
-0.8199 -0.5807
-0.8199 -0.5807
-0.8199 -0.5807
[torch.DoubleTensor of size 4x2]

prev_h	{
  1 : DoubleTensor - size: 4x10
  2 : DoubleTensor - size: 4x4
  3 : DoubleTensor - size: 4x4
}


RNN3 output	
y	-0.8215 -0.5794
-0.8215 -0.5794
-0.8215 -0.5794
-0.8215 -0.5794
[torch.DoubleTensor of size 4x2]

h	{
  1 : DoubleTensor - size: 4x10
  2 : DoubleTensor - size: 4x4
  3 : DoubleTensor - size: 4x4
}
==================================> RNN t =	6	
Recurrent type(inp) ==	table	
Recurrent inp = 	{
  1 : DoubleTensor - size: 4x10x5
  2 : DoubleTensor - size: 4x10x6
}
Recurrent	
inp	{
  1 : DoubleTensor - size: 4x10x5
  2 : DoubleTensor - size: 4x10x6
}
prev_y	-0.8215 -0.5794
-0.8215 -0.5794
-0.8215 -0.5794
-0.8215 -0.5794
[torch.DoubleTensor of size 4x2]

prev_h	{
  1 : DoubleTensor - size: 4x10
  2 : DoubleTensor - size: 4x4
  3 : DoubleTensor - size: 4x4
}
RNN3 output	
y	-0.8227 -0.5785
-0.8227 -0.5785
-0.8227 -0.5785
-0.8227 -0.5785
[torch.DoubleTensor of size 4x2]

h	{
  1 : DoubleTensor - size: 4x10
  2 : DoubleTensor - size: 4x4
  3 : DoubleTensor - size: 4x4
}
==================================> RNN t =	7	
Recurrent type(inp) ==	table	
Recurrent inp = 	{
  1 : DoubleTen

...

LSTM	
inp	 1.6935 -0.1168  1.2906  1.8172
 1.6935 -0.1168  1.2906  1.8172
 1.6935 -0.1168  1.2906  1.8172
 1.6935 -0.1168  1.2906  1.8172
[torch.DoubleTensor of size 4x4]

prev_h	-0.2157  0.1629 -0.1683 -0.1979
-0.2157  0.1629 -0.1683 -0.1979
-0.2157  0.1629 -0.1683 -0.1979
-0.2157  0.1629 -0.1683 -0.1979
[torch.DoubleTensor of size 4x4]

prev_c	-0.5060  0.2716 -0.5063 -0.5653
-0.5060  0.2716 -0.5063 -0.5653
-0.5060  0.2716 -0.5063 -0.5653
-0.5060  0.2716 -0.5063 -0.5653
[torch.DoubleTensor of size 4x4]



LSTM	
inp	 1.6932 -0.1164  1.2907  1.8166
 1.6932 -0.1164  1.2907  1.8166
 1.6932 -0.1164  1.2907  1.8166
 1.6932 -0.1164  1.2907  1.8166
[torch.DoubleTensor of size 4x4]

prev_h	-0.2118  0.1436 -0.1600 -0.1811
-0.2118  0.1436 -0.1600 -0.1811
-0.2118  0.1436 -0.1600 -0.1811
-0.2118  0.1436 -0.1600 -0.1811
[torch.DoubleTensor of size 4x4]

prev_c	-0.4937  0.2393 -0.4738 -0.5059
-0.4937  0.2393 -0.4738 -0.5059
-0.4937  0.2393 -0.4738 -0.5059
-0.4937  0.2393 -0.4738 -0.5059
[torch.DoubleTensor of size 4x4]

LSTM	
inp	 1.6927 -0.1158  1.2908  1.8159
 1.6927 -0.1158  1.2908  1.8159
 1.6927 -0.1158  1.2908  1.8159
 1.6927 -0.1158  1.2908  1.8159
[torch.DoubleTensor of size 4x4]

prev_h	-0.2034  0.1193 -0.1469 -0.1602
-0.2034  0.1193 -0.1469 -0.1602
-0.2034  0.1193 -0.1469 -0.1602
-0.2034  0.1193 -0.1469 -0.1602
[torch.DoubleTensor of size 4x4]

prev_c	-0.4694  0.1991 -0.4266 -0.4367
-0.4694  0.1991 -0.4266 -0.4367
-0.4694  0.1991 -0.4266 -0.4367
-0.4694  0.1991 -0.4266 -0.4367
[torch.DoubleT

LSTM	
inp	 1.6921 -0.1151  1.2909  1.8147
 1.6921 -0.1151  1.2909  1.8147
 1.6921 -0.1151  1.2909  1.8147
 1.6921 -0.1151  1.2909  1.8147
[torch.DoubleTensor of size 4x4]

prev_h	-0.1868  0.0889 -0.1268 -0.1333
-0.1868  0.0889 -0.1268 -0.1333
-0.1868  0.0889 -0.1268 -0.1333
-0.1868  0.0889 -0.1268 -0.1333
[torch.DoubleTensor of size 4x4]

prev_c	-0.4248  0.1494 -0.3598 -0.3529
-0.4248  0.1494 -0.3598 -0.3529
-0.4248  0.1494 -0.3598 -0.3529
-0.4248  0.1494 -0.3598 -0.3529
[torch.DoubleTensor of size 4x4]

LSTM	
inp	 1.6911 -0.1139  1.2911  1.8131
 1.6911 -0.1139  1.2911  1.8131
 1.6911 -0.1139  1.2911  1.8131
 1.6911 -0.1139  1.2911  1.8131
[torch.DoubleTensor of size 4x4]

prev_h	-0.1555  0.0521 -0.0965 -0.0962
-0.1555  0.0521 -0.0965 -0.0962
-0.1555  0.0521 -0.0965 -0.0962
-0.1555  0.0521 -0.0965 -0.0962
[torch.DoubleTensor of size 4x4]

prev_c	-0.3460  0.0889 -0.2674 -0.2458
-0.3460  0.0889 -0.2674 -0.2458
-0.3460  0.0889 -0.2674 -0.2458
-0.3460  0.0889 -0.2674 -0.2458
[torch.DoubleT

LSTM	
inp	 1.6895 -0.1121  1.2913  1.8105
 1.6895 -0.1121  1.2913  1.8105
 1.6895 -0.1121  1.2913  1.8105
 1.6895 -0.1121  1.2913  1.8105
[torch.DoubleTensor of size 4x4]

prev_h	0.01 *
 -9.2555  0.9338 -5.3572 -4.5063
 -9.2555  0.9338 -5.3572 -4.5063
 -9.2555  0.9338 -5.3572 -4.5063
 -9.2555  0.9338 -5.3572 -4.5063
[torch.DoubleTensor of size 4x4]

prev_c	-0.2114  0.0170 -0.1436 -0.0974
-0.2114  0.0170 -0.1436 -0.0974
-0.2114  0.0170 -0.1436 -0.0974
-0.2114  0.0170 -0.1436 -0.0974
[torch.DoubleTensor of size 4x4]

LSTM	
inp	 1.6365  0.1245  0.9352  1.3752
 1.6365  0.1245  0.9352  1.3752
 1.6365  0.1245  0.9352  1.3752
 1.6365  0.1245  0.9352  1.3752
[torch.DoubleTensor of size 4x4]

prev_h	

 0  0  0  0
 0  0  0  0
 0  0  0  0
 0  0  0  0
[torch.DoubleTensor of size 4x4]

prev_c	 0  0  0  0
 0  0  0  0
 0  0  0  0
 0  0  0  0
[torch.DoubleTensor of size 4x4]



In [4]:
function customToDot(graph, title, failedNode)
   local str = graph:todot(title)
   if not failedNode then
      return str
   end

   local failedNodeId = nil
   for i, node in ipairs(graph.nodes) do
      if node.data == failedNode.data then
         failedNodeId = node.id
         break
      end
   end

   if failedNodeId ~= nil then
      -- The closing '}' is removed.
      -- And red fillcolor is specified for the failedNode.
      str = string.gsub(str, '}%s*$', '')
      str = str .. string.format('n%s[style=filled, fillcolor=red];\n}',
      failedNodeId)
   end
   return str
end

function saveSvg(svgPathPrefix, dotStr)
   io.stderr:write(string.format("saving %s.svg\n", svgPathPrefix))
   local dotPath = svgPathPrefix .. '.dot'
   local dotFile = io.open(dotPath, 'w')
   dotFile:write(dotStr)
   dotFile:close()

   local svgPath = svgPathPrefix .. '.svg'
   local cmd = string.format('dot -Tsvg -o %s %s', svgPath, dotPath)
   os.execute(cmd)
end

function outputGraphViz(gmodule, focusNode)
   local focusNode = focusNode or gmodule.outnode
   local nInputs = gmodule.nInputs or #gmodule.innode.children
   local svgPathPrefix = gmodule.name or string.format(
   'nngraph_%sin_%sout', nInputs, #gmodule.outnode.children)
   local dotStr = customToDot(gmodule.fg, svgPathPrefix, focusNode)
   saveSvg(svgPathPrefix, dotStr)
end

In [5]:
analyze_graph = m.rnn.data.module.rnn[1].recurrent
outputGraphViz(analyze_graph.forwardnodes[41],analyze_graph)

In [13]:
analyze_graph..o.forwardnodes[41]

nngraph.Node
{
  data : 
    {
      annotations : 
        {
          _debugLabel : [[C]]:-1
          graphAttributes : 
            {
              tooltip : [[C]]:-1
            }
        }
      mapindex : 
        {
          1 : 
            {
              annotations : 
                {
                  _debugLabel : [./Attention.lua]:115
                  graphAttributes : table: 0x0fbbf300
                }
              input : 
                {
                  1 : DoubleTensor - size: 1x4
                  2 : DoubleTensor - size: 1x6
                }
              module : 
                decoder_mlp
                {
                  bg : graph.Graph
                  forwardnodes : table: 0x0fd2a528
                  outnode : nngraph.Node
                  fg : graph.Graph
                  verbose : false
                  backwardnodes : table: 0x0fd2fee8
                  output : DoubleTensor - size: 1x2
                  innode : nngraph.Node
            

In [20]:
V = nn.View(-1)
V:setNumInputDims(2)
print(V:forward(torch.Tensor(1,6)))
print(V:forward(torch.Tensor(2,1,6)))

  2.3158e+77
  2.3158e+77
  2.3278e-57
  1.4270e-71
 1.4627e+165
 4.3579e-309
[torch.DoubleTensor of size 6]

  2.3158e+77   2.3158e+77  1.7055e+256  1.7572e+243  9.9592e-143  7.7511e+228
 7.7515e+228  5.5622e+180  2.9791e+228  9.4487e-143  3.9708e+246  2.7762e+184
[torch.DoubleTensor of size 2x6]



In [13]:
nn.Reshape(1,10):forward(torch.ones(20,10)):size()


 20
  1
 10
[torch.LongStorage of size 3]



In [14]:
a[1] = nil
a[2] = nil

In [15]:
a

{
  0 : 1
}


In [27]:
inp = {u,p,2}
a,b,c = unpack(inp)

In [30]:
c

2	


In [6]:
#torch.Tensor(1) == 1

false	


In [31]:
a = {2,3}

function recursiveOperation(x,func)
    local output
    if type(x) == 'table' then
        output = {}
        for i = 1, #x do
            output[i] = recursiveOperation(x[i],func)
        end
    else
        output = func(x)
    end
    return output
end


myfunc = function(x) return torch.Tensor(x) end

b = recursiveOperation(a,myfunc)

In [32]:
b

{
  1 : DoubleTensor - size: 2
  2 : DoubleTensor - size: 3
}


In [13]:
a = nn.Identity()()
b = nn.Identity()()

c = nn.Identity()({a,b})
--d = nn.JoinTable(1,1)(c)

e = nn.gModule({a,b},{c})

In [14]:
e:forward({torch.ones(2),torch.zeros(3)})

{
  1 : DoubleTensor - size: 2
  2 : DoubleTensor - size: 3
}


In [16]:
x = torch.zeros(3,4)
dim = {5,3,4}
print(x:|resize(unpack(dim)))

[string "x = torch.zeros(3,4)..."]:3: '<name>' expected near '|': 

In [1]:
x = torch.zeros(3,4)

In [2]:
x

 0  0  0  0
 0  0  0  0
 0  0  0  0
[torch.DoubleTensor of size 3x4]



In [3]:
x:resize(3,4)

 0  0  0  0
 0  0  0  0
 0  0  0  0
[torch.DoubleTensor of size 3x4]



In [4]:
x:resize(3,4)

 0  0  0  0
 0  0  0  0
 0  0  0  0
[torch.DoubleTensor of size 3x4]



In [5]:
x:resize(3,4)

 0  0  0  0
 0  0  0  0
 0  0  0  0
[torch.DoubleTensor of size 3x4]



In [6]:
b = {}
a = torch.zeros(2)
b[1] = a
a = torch.ones(3)
b[2] = a
print(b)

{
  1 : DoubleTensor - size: 2
  2 : DoubleTensor - size: 3
}


In [7]:
b[1]

 0
 0
[torch.DoubleTensor of size 2]



In [8]:
b[2]

 1
 1
 1
[torch.DoubleTensor of size 3]



In [10]:
function resizeGradInput(input,gradInput)
    if type(input) == 'table' then
        if type(gradInput) ~= 'table' then
            gradInput = {}
        end
        for i = 1, #input do
            gradInput[i] = resizeGradInput(input[i],gradInput[i])
        end
    else
        gradInput = gradInput or input.new()
        gradInput:resizeAs(input):zero()
    end
    return gradInput
end

In [11]:
input = torch.Tensor(3,4)
gradInput = torch.Tensor(3,4)
print(resizeGradInput(input,gradInput))

 0  0  0  0
 0  0  0  0
 0  0  0  0
[torch.DoubleTensor of size 3x4]



In [12]:
input = torch.Tensor(3,4)
gradInput = nil
print(resizeGradInput(input,gradInput))

 0  0  0  0
 0  0  0  0
 0  0  0  0
[torch.DoubleTensor of size 3x4]



In [13]:
input = {torch.Tensor(3,4)}
gradInput = torch.Tensor(3,4)
print(resizeGradInput(input,gradInput))

{
  1 : DoubleTensor - size: 3x4
}


In [14]:
input = {torch.Tensor(3,4)}
gradInput = nil
print(resizeGradInput(input,gradInput))

{
  1 : DoubleTensor - size: 3x4
}


In [15]:
input = {torch.Tensor(3,4),torch.Tensor(2,3)}
gradInput = nil
print(resizeGradInput(input,gradInput))

{
  1 : DoubleTensor - size: 3x4
  2 : DoubleTensor - size: 2x3
}


In [16]:
input = {torch.Tensor(3,4),torch.Tensor(2,3)}
gradInput = {torch.Tensor(3,4),torch.Tensor(2,3)}
print(resizeGradInput(input,gradInput))

{
  1 : DoubleTensor - size: 3x4
  2 : DoubleTensor - size: 2x3
}


In [19]:
input = {torch.Tensor(3,4),torch.Tensor(2,3)}
gradInput = {torch.ones(3,3,4),torch.zeros(3,2,3)}
print(resizeGradInput(input,gradInput))
print(resizeGradInput(input,gradInput)[1])
print(resizeGradInput(input,gradInput)[2])

{
  1 : DoubleTensor - size: 3x4
  2 : DoubleTensor - size: 2x3
}
 0  0  0  0
 0  0  0  0
 0  0  0  0
[torch.DoubleTensor of size 3x4]

 0  0  0
 0  0  0
[torch.DoubleTensor of size 2x3]



In [20]:
input = {torch.Tensor(3,4),{torch.Tensor(1,2),torch.Tensor(2,3)}}
gradInput = {torch.ones(3,3,4),torch.zeros(3,2,3)}
print(resizeGradInput(input,gradInput))
print(resizeGradInput(input,gradInput)[1])
print(resizeGradInput(input,gradInput)[2])

{
  1 : DoubleTensor - size: 3x4
  2 : 
    {
      1 : DoubleTensor - size: 1x2
      2 : DoubleTensor - size: 2x3
    }
}
 0  0  0  0
 0  0  0  0
 0  0  0  0
[torch.DoubleTensor of size 3x4]

{
  1 : DoubleTensor - size: 1x2
  2 : DoubleTensor - size: 2x3
}
